In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import os
import datetime
pd.options.display.max_rows = 20

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

import warnings
warnings.filterwarnings(action='ignore')

## 아래는 matplotlib 한국어 표시해주는 코드

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

# 전처리

## 생활인구

### 코드파일 처리

In [2]:
dcode = pd.read_excel('./data/행정동코드.xlsx',encoding='cp949')


In [3]:
dcode.head()

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,1100000000,서울특별시,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,1111000000,종로구,19880423,NaN
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,NaN
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,NaN
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,NaN


In [4]:
dcode = dcode.iloc[:,0:4] #필요없는 column삭제
dcode = dcode[dcode.시도명 =="서울특별시"] #서울특별시만 데려온다
dcode = dcode.drop_duplicates() #중복삭제
dcode.head()

,행정동코드,시도명,시군구명,읍면동명
0,1100000000,서울특별시,NaN,NaN
1,1111000000,서울특별시,종로구,NaN
2,1111051500,서울특별시,종로구,청운효자동
12,1111053000,서울특별시,종로구,사직동
24,1111054000,서울특별시,종로구,삼청동


In [5]:
dcode.columns = ["code","시도명","구","동"] #column명 내가 원하는 대로 바꾸기
dcode['code'] = dcode['code'].map(lambda x : str(x)[:-2]) #생활인구파일에 맞게 코드 조정
dcode.head()

,code,시도명,구,동
0,11000000,서울특별시,NaN,NaN
1,11110000,서울특별시,종로구,NaN
2,11110515,서울특별시,종로구,청운효자동
12,11110530,서울특별시,종로구,사직동
24,11110540,서울특별시,종로구,삼청동


### 생활인구파일 : 일별로 합산

In [6]:
def people_count_sum(df):    
    df = df.reset_index().iloc[:,0:4]
    df.columns = ["날짜","hour","code","생활인구수"]
    #df.drop(['hour'], axis = 1,inplace = True)
    df_return = df.groupby(['날짜', 'code']).apply(func) #날짜별, 코드별 sum
    df2 = df_return.reset_index()
    df2['code'] = df2['code'].map(lambda x : str(x))
    res = pd.merge(df2,dcode,on="code")
    res.drop(['code'],axis = 1, inplace = True)
    return res

In [7]:
def func(x):
    d={}
    d['생활인구수'] = (x['생활인구수'].sum())/24

    return pd.Series(d, index=['생활인구수'])

### 하나만 미리 해보기

In [14]:
data = pd.read_csv(f'./data/LOCAL_PEOPLE_DONG_202002.csv',encoding ='utf-8')   

In [15]:
data.head()

,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
20200201,0,11440730,40008.7419,1346.3171,841.9587,1061.1255,1050.6067,1347.5052,1493.9572,1755.6993,...,1937.7415,1978.4616,1743.0507,1825.5686,1429.1436,1257.2309,1227.2301,958.5541,2808.6218,NaN
20200201,0,11650621,26212.2534,789.3616,519.2963,864.9748,627.7166,837.1151,905.2537,956.8247,...,995.9951,1162.6565,1138.2961,1314.2640,991.8098,948.0337,918.3349,666.0662,1971.8968,NaN
20200201,0,11215830,23724.4049,762.3188,356.6239,562.7924,617.7153,875.1754,1006.7162,1064.8150,...,1147.8351,1191.2545,915.0628,902.7632,902.2134,1049.6805,904.8118,644.3256,1455.1883,NaN
20200201,0,11290770,14879.6820,554.3716,386.5561,318.8035,397.4454,476.9146,386.7727,521.9637,...,367.2178,508.3087,494.4807,667.0206,582.6305,637.4483,593.1703,475.5271,1315.0792,NaN
20200201,0,11305555,30307.1826,998.5169,552.0452,733.2881,732.6588,914.3310,881.1347,1018.6746,...,1002.1755,1096.1847,1041.6831,1303.0572,1241.5933,1328.6948,1201.8803,947.3990,3024.0821,NaN


In [16]:
data.columns

Index(['기준일ID', '시간대구분', '행정동코드', '총생활인구수', '남자0세부터9세생활인구수', '남자10세부터14세생활인구수',
       '남자15세부터19세생활인구수', '남자20세부터24세생활인구수', '남자25세부터29세생활인구수',
       '남자30세부터34세생활인구수', '남자35세부터39세생활인구수', '남자40세부터44세생활인구수',
       '남자45세부터49세생활인구수', '남자50세부터54세생활인구수', '남자55세부터59세생활인구수',
       '남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '남자70세이상생활인구수', '여자0세부터9세생활인구수',
       '여자10세부터14세생활인구수', '여자15세부터19세생활인구수', '여자20세부터24세생활인구수',
       '여자25세부터29세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수',
       '여자40세부터44세생활인구수', '여자45세부터49세생활인구수', '여자50세부터54세생활인구수',
       '여자55세부터59세생활인구수', '여자60세부터64세생활인구수', '여자65세부터69세생활인구수',
       '여자70세이상생활인구수'],
      dtype='object')

In [17]:
data = data.reset_index().iloc[:,:4] #필요한 열만 남기기
data.columns = ["날짜","hour","code","생활인구수"]


In [18]:
data.head()

,날짜,hour,code,생활인구수
0,20200201,0,11440730,40008.7419
1,20200201,0,11650621,26212.2534
2,20200201,0,11215830,23724.4049
3,20200201,0,11290770,14879.6820
4,20200201,0,11305555,30307.1826


In [19]:
df_return = data.groupby(['날짜', 'code']).apply(func) #날짜,구끼리 그룹화하고, 생활인구수 합하기(func)

In [20]:
df_return.head()

생활인구수
날짜       code                  
20200201 11110515  15331.758558
         11110530  22442.245846
         11110540   5479.183912
         11110550  15573.699683
         11110560  18687.032317

In [21]:
df2 = df_return.reset_index() #index정리
df2.head()

,날짜,code,생활인구수
0,20200201,11110515,15331.758558
1,20200201,11110530,22442.245846
2,20200201,11110540,5479.183912
3,20200201,11110550,15573.699683
4,20200201,11110560,18687.032317


In [22]:
 #str으로 바꿔서 dcode['code']와 데이터타입맞추기
df2['code'] = df2['code'].map(lambda x : str(x))

#merge로 join. key값은 code
res = pd.merge(df2,dcode,on="code")
res.head()

,날짜,code,생활인구수,시도명,구,동
0,20200201,11110515,15331.758558,서울특별시,종로구,청운효자동
1,20200202,11110515,14910.355892,서울특별시,종로구,청운효자동
2,20200203,11110515,16006.643896,서울특별시,종로구,청운효자동
3,20200204,11110515,16185.131108,서울특별시,종로구,청운효자동
4,20200205,11110515,16516.664483,서울특별시,종로구,청운효자동


In [23]:
#code column 삭제
res.drop(['code'],axis = 1, inplace = True)
res.head()

,날짜,생활인구수,시도명,구,동
0,20200201,15331.758558,서울특별시,종로구,청운효자동
1,20200202,14910.355892,서울특별시,종로구,청운효자동
2,20200203,16006.643896,서울특별시,종로구,청운효자동
3,20200204,16185.131108,서울특별시,종로구,청운효자동
4,20200205,16516.664483,서울특별시,종로구,청운효자동


### 여러개 한 번에

In [24]:
list_ = []
for k in range(2,9):
    data = pd.read_csv(f'./data/LOCAL_PEOPLE_DONG_20200{k}.csv',encoding ='utf-8')
    df = people_count_sum(data)
    list_.append(df)

In [25]:
#19년
for k in range(2,9): 
    data = pd.read_csv(f'./data/LOCAL_PEOPLE_DONG_20190{k}.csv',encoding ='utf-8')
    df = people_count_sum(data)
    list_.append(df)

### '구별'로 묶기

In [26]:
def func_dong(x):
    d={}
    d['생활인구수'] = (x['생활인구수'].sum())

    return pd.Series(d, index=['생활인구수'])

In [27]:
res = pd.concat(list_)
res.head()

,날짜,생활인구수,시도명,구,동
0,20200201,15331.758558,서울특별시,종로구,청운효자동
1,20200202,14910.355892,서울특별시,종로구,청운효자동
2,20200203,16006.643896,서울특별시,종로구,청운효자동
3,20200204,16185.131108,서울특별시,종로구,청운효자동
4,20200205,16516.664483,서울특별시,종로구,청운효자동


In [28]:
res.tail()

,날짜,생활인구수,시도명,구,동
13139,20190827,32314.247254,서울특별시,강동구,둔촌제2동
13140,20190828,32128.861117,서울특별시,강동구,둔촌제2동
13141,20190829,31886.786246,서울특별시,강동구,둔촌제2동
13142,20190830,31552.028246,서울특별시,강동구,둔촌제2동
13143,20190831,30241.320771,서울특별시,강동구,둔촌제2동


In [29]:
res_return = res.groupby(['날짜', '구']).apply(func_dong) #동 합치기
res1 = res_return.reset_index()
res1.head(3)

,날짜,구,생활인구수
0,20190201,강남구,852099.178121
1,20190201,강동구,492496.122871
2,20190201,강북구,302984.858925


In [30]:
res1.tail()

,날짜,구,생활인구수
10620,20200831,용산구,282195.691312
10621,20200831,은평구,449585.908367
10622,20200831,종로구,272909.327746
10623,20200831,중구,267535.523779
10624,20200831,중랑구,359603.299646


### 저장

In [ ]:
res.to_csv('생활인구_전처리.csv',encoding='cp949') #동포함

In [ ]:
res1.to_csv('생활인구_전처리_구.csv',encoding='cp949') #구까지만

## 유동인구

In [31]:
def yd_sum(yd):
    yd.drop(['시간(1시간단위)','연령대(10세단위)','성별'],axis = 1, inplace = True)
    yd.columns = ["날짜","시","구","유동인구수"]
    yd_return = yd.groupby(['날짜', '구']).apply(func_yd)
    yd2 = yd_return.reset_index()
    return yd2

In [32]:
def func_yd(x):
    d={}
    d['유동인구수'] = (x['유동인구수'].sum())/24

    return pd.Series(d, index=['유동인구수'])

In [33]:
list_yd = []
for k in range(2,9):
    data = pd.read_csv(f'./data/Floating_Population_200{k}.csv',encoding ='utf-8')
    yd = yd_sum(data)
    list_yd.append(yd)

In [ ]:
#19년
#for k in range(3,9):
#    data = pd.read_csv(f'./data/FLT_SEOUL_0{k}MONTH.csv',encoding='utf-8')
#    yd = yd_sum(data)
#    list_yd.append(yd)

In [34]:
res2 = pd.concat(list_yd)
res2

,날짜,구,유동인구수
0,20200201,강남구,526535.000000
1,20200201,강동구,347490.416667
2,20200201,강북구,248604.166667
3,20200201,강서구,452703.333333
4,20200201,관악구,415262.500000
...,...,...,...
770,20200831,용산구,212702.916667
771,20200831,은평구,332800.416667
772,20200831,종로구,214046.666667
773,20200831,중구,230347.500000


In [ ]:
res2.to_csv('유동인구_전처리.csv',encoding='cp949')

## 생활/유동 합치기

In [35]:
sum_all = pd.merge(res1, res2, how='outer', on = ["날짜","구"])
sum_all

,날짜,구,생활인구수,유동인구수
0,20190201,강남구,852099.178121,NaN
1,20190201,강동구,492496.122871,NaN
2,20190201,강북구,302984.858925,NaN
3,20190201,강서구,557574.586904,NaN
4,20190201,관악구,494731.264333,NaN
...,...,...,...,...
10620,20200831,용산구,282195.691312,212702.916667
10621,20200831,은평구,449585.908367,332800.416667
10622,20200831,종로구,272909.327746,214046.666667
10623,20200831,중구,267535.523779,230347.500000


In [ ]:
sum_all.to_csv('생활유동합침.csv',encoding='cp949')

## 코로나 확진자 추가

In [36]:
data1 = pd.read_csv('./data/코로나19발생현황.csv', encoding='cp949')
data2 = pd.read_csv('./data/FINAL_FINAL_코로나추가.csv', encoding='utf-8')

### 전국확진자 신규/누적 추가

In [37]:
data_sum = pd.merge(data2,data1,on="날짜",how="inner")
data_sum.head()

,Unnamed: 0,날짜,구,생활인구수,유동인구수,전체점포수,프랜차이즈점포수,일반점포수,확진자수,신규,누적
0,0,2020-02-01,강남구,759286.0842,526535.0000,55875,4371,51504,0,0,0
1,1,2020-02-01,강동구,519147.6209,347490.4167,20974,1901,19073,0,0,0
2,2,2020-02-01,강북구,319527.8081,248604.1667,13417,1075,12342,0,0,0
3,3,2020-02-01,강서구,555865.8080,452703.3333,25902,2633,23269,0,0,0
4,4,2020-02-01,관악구,520445.2208,415262.5000,19734,1908,17826,0,0,0


In [38]:
data_sum.to_csv('FINAL_FINAL_코로나추가.csv',encoding='cp949')

In [39]:
covid = pd.read_csv('./data/코로나서울확진자.csv',encoding='cp949')
covid.tail(8)

,환자번호,감염경로,확진일,거주지,퇴원현황
5455,13,해외 접촉(우한 교민),2020-02-02,타시도,퇴원
5456,11,#6 접촉,2020-01-31,종로구,퇴원
5457,10,#6 접촉,2020-01-31,종로구,퇴원
5458,9,#5 접촉,2020-01-31,성북구,퇴원
5459,7,해외 접촉(중국 우한시),2020-01-30,마포구,퇴원
5460,6,#3 접촉,2020-01-30,종로구,퇴원
5461,5,해외 접촉(중국 우한시),2020-01-30,중랑구,퇴원
5462,2,해외 접촉(중국 우한시),2020-01-24,강서구,퇴원


In [40]:
def func_covid(x):
    d={}
    d['서울신규'] = (x['환자번호'].count())

    return pd.Series(d, index=['서울신규'])

In [41]:
covid.tail()

,환자번호,감염경로,확진일,거주지,퇴원현황
5458,9,#5 접촉,2020-01-31,성북구,퇴원
5459,7,해외 접촉(중국 우한시),2020-01-30,마포구,퇴원
5460,6,#3 접촉,2020-01-30,종로구,퇴원
5461,5,해외 접촉(중국 우한시),2020-01-30,중랑구,퇴원
5462,2,해외 접촉(중국 우한시),2020-01-24,강서구,퇴원


In [42]:
covid2 = covid.groupby(['확진일']).apply(func_covid)

In [43]:
covid2 = covid2.reset_index()
covid2.columns = ["날짜","서울신규"]

In [44]:
covid2["서울누적"] = covid2["서울신규"].cumsum()
covid2.tail()

,날짜,서울신규,서울누적
224,2020-10-03,16,5376
225,2020-10-04,19,5395
226,2020-10-05,18,5413
227,2020-10-06,30,5443
228,2020-10-07,20,5463


In [45]:
final_ = pd.read_csv('FINAL_FINAL_코로나추가.csv',encoding='cp949')
final_.tail()

,Unnamed: 0,Unnamed: 0.1,날짜,구,생활인구수,유동인구수,전체점포수,프랜차이즈점포수,일반점포수,확진자수,신규,누적
5334,5334,5334,2020-08-31,은평구,449585.9084,332800.4167,18350,1572,16778,3,248,19947
5335,5335,5335,2020-08-31,종로구,272909.3277,214046.6667,25318,1608,23710,2,248,19947
5336,5336,5336,2020-08-31,중구,267535.5238,230347.5000,40015,2062,37953,0,248,19947
5337,5337,5337,2020-08-31,중랑구,359603.2996,293332.0833,16180,1286,14894,6,248,19947
5338,5338,5338,2020-08-31,중랑구,359603.2996,293332.0833,16180,1286,14894,4,248,19947


In [46]:
final = pd.merge(final_,covid2,on="날짜", how="left")
final.tail()

,Unnamed: 0,Unnamed: 0.1,날짜,구,생활인구수,유동인구수,전체점포수,프랜차이즈점포수,일반점포수,확진자수,신규,누적,서울신규,서울누적
5334,5334,5334,2020-08-31,은평구,449585.9084,332800.4167,18350,1572,16778,3,248,19947,104.0,4019.0
5335,5335,5335,2020-08-31,종로구,272909.3277,214046.6667,25318,1608,23710,2,248,19947,104.0,4019.0
5336,5336,5336,2020-08-31,중구,267535.5238,230347.5000,40015,2062,37953,0,248,19947,104.0,4019.0
5337,5337,5337,2020-08-31,중랑구,359603.2996,293332.0833,16180,1286,14894,6,248,19947,104.0,4019.0
5338,5338,5338,2020-08-31,중랑구,359603.2996,293332.0833,16180,1286,14894,4,248,19947,104.0,4019.0


In [47]:
final.isnull().sum()

Unnamed: 0        0
Unnamed: 0.1      0
날짜                0
구                 0
생활인구수             0
유동인구수             0
전체점포수             0
프랜차이즈점포수          0
일반점포수             0
확진자수              0
신규                0
누적                0
서울신규            600
서울누적            600
dtype: int64

In [48]:
final["서울신규"] = final["서울신규"].fillna(0)
final["서울누적"] = final["서울누적"].fillna(method = 'ffill')
final["서울누적"] = final["서울누적"].fillna(7)
final.head(2)

,Unnamed: 0,Unnamed: 0.1,날짜,구,생활인구수,유동인구수,전체점포수,프랜차이즈점포수,일반점포수,확진자수,신규,누적,서울신규,서울누적
0,0,0,2020-02-01,강남구,759286.0842,526535.0000,55875,4371,51504,0,0,0,0.0,7.0
1,1,1,2020-02-01,강동구,519147.6209,347490.4167,20974,1901,19073,0,0,0,0.0,7.0


In [49]:
final.drop(['Unnamed: 0.1'],axis = 1, inplace = True)
final.tail()

,Unnamed: 0,날짜,구,생활인구수,유동인구수,전체점포수,프랜차이즈점포수,일반점포수,확진자수,신규,누적,서울신규,서울누적
5334,5334,2020-08-31,은평구,449585.9084,332800.4167,18350,1572,16778,3,248,19947,104.0,4019.0
5335,5335,2020-08-31,종로구,272909.3277,214046.6667,25318,1608,23710,2,248,19947,104.0,4019.0
5336,5336,2020-08-31,중구,267535.5238,230347.5000,40015,2062,37953,0,248,19947,104.0,4019.0
5337,5337,2020-08-31,중랑구,359603.2996,293332.0833,16180,1286,14894,6,248,19947,104.0,4019.0
5338,5338,2020-08-31,중랑구,359603.2996,293332.0833,16180,1286,14894,4,248,19947,104.0,4019.0


In [ ]:
final.to_csv('FINAL_FINAL_코로나추가2.csv',encoding='cp949')

In [50]:
final.isnull().sum()

Unnamed: 0    0
날짜            0
구             0
생활인구수         0
유동인구수         0
전체점포수         0
프랜차이즈점포수      0
일반점포수         0
확진자수          0
신규            0
누적            0
서울신규          0
서울누적          0
dtype: int64

# 

## 생활인구

In [52]:
prep_col = ['m0', 'm10',
       'm15', 'm20','m25','m30','m35','m40','m45','m50','m55','m60','m65','m70','f0', 'f10',
       'f15', 'f20','f25','f30','f35','f40','f45','f50','f55','f60','f65','f70']

In [53]:
def func_all(x):
    d={}
    for k in prep_col:
        d[k] = (x[k].sum())/24
    return pd.Series(d, index=prep_col)

In [54]:
def all_dong(x):
    d={}
    for k in prep_col:
        d[k] = (x[k].sum())
    return pd.Series(d, index=prep_col)
    

In [55]:
def sum_all_people(df):    
    df = df.reset_index().iloc[:,:32]
    df.columns = ["날짜","hour","code","생활인구수",'m0', 'm10',
       'm15', 'm20','m25','m30','m35','m40','m45','m50','m55','m60','m65','m70','f0', 'f10',
       'f15', 'f20','f25','f30','f35','f40','f45','f50','f55','f60','f65','f70']
    #df.drop(['hour'], axis = 1,inplace = True)
    df_return = df.groupby(['날짜', 'code']).apply(func_all) #날짜별, 코드별 sum
    df2 = df_return.reset_index()
    df2['code'] = df2['code'].map(lambda x : str(x))
    res = pd.merge(df2,dcode,on="code")
    res.drop(['code'],axis = 1, inplace = True)
    return res

In [56]:
list_all = []
for k in range(2,9):
    data = pd.read_csv(f'./data/LOCAL_PEOPLE_DONG_20200{k}.csv',encoding ='utf-8')
    df = sum_all_people(data)
    list_all.append(df)

In [57]:
for k in range(3,9):
    data = pd.read_csv(f'./data/LOCAL_PEOPLE_DONG_20190{k}.csv',encoding='utf-8')
    df = sum_all_people(data)
    list_all.append(df)

In [58]:
sh = pd.concat(list_all)

In [66]:
sh.sample()

,날짜,m0,m10,m15,m20,m25,m30,m35,m40,m45,...,f40,f45,f50,f55,f60,f65,f70,시도명,구,동
3680,20190723,891.237392,467.012608,585.738871,582.746383,519.191946,513.856396,647.283196,608.188533,744.18525,...,792.387038,835.846013,758.461033,774.987942,775.277346,543.706621,1822.678233,서울특별시,성북구,정릉제2동


In [60]:
sh_return = sh.groupby(['날짜', '구']).apply(all_dong) #동 합치기
sh1 = sh_return.reset_index()
sh1.head(3)

,날짜,구,m0,m10,m15,m20,m25,m30,m35,m40,...,f25,f30,f35,f40,f45,f50,f55,f60,f65,f70
0,20190301,강남구,21298.583608,13923.692279,22074.344375,21395.810725,30065.875163,30457.165417,33868.153392,31024.321754,...,34867.720592,32025.190200,33017.016846,29807.846417,31308.189150,22593.272658,23724.084492,22808.293246,17463.134562,41000.019713
1,20190301,강동구,19973.502550,10664.142133,13796.594988,13158.594667,15437.459538,15841.211633,19582.164629,18176.002796,...,15564.845429,16166.849892,19892.289692,18092.193188,20302.464463,18530.740029,22348.468150,20536.102179,15101.367063,33649.091133
2,20190301,강북구,9880.497638,5431.196442,7283.578792,9711.192017,10428.617500,9253.645213,11174.490771,10763.451583,...,10341.266958,8887.088354,10767.592033,10659.554071,13129.848975,12195.954863,13948.280575,12592.444421,10638.855375,30880.407996


In [62]:
res1.tail()

,날짜,구,생활인구수
10620,20200831,용산구,282195.691312
10621,20200831,은평구,449585.908367
10622,20200831,종로구,272909.327746
10623,20200831,중구,267535.523779
10624,20200831,중랑구,359603.299646


In [65]:
sh_all = pd.merge(res1, sh1, on = ["날짜","구"])
sh_all

,날짜,구,생활인구수,m0,m10,m15,m20,m25,m30,m35,...,f25,f30,f35,f40,f45,f50,f55,f60,f65,f70
0,20190301,강남구,713865.351558,21298.583608,13923.692279,22074.344375,21395.810725,30065.875163,30457.165417,33868.153392,...,34867.720592,32025.190200,33017.016846,29807.846417,31308.189150,22593.272658,23724.084492,22808.293246,17463.134562,41000.019713
1,20190301,강동구,484874.103471,19973.502550,10664.142133,13796.594988,13158.594667,15437.459538,15841.211633,19582.164629,...,15564.845429,16166.849892,19892.289692,18092.193188,20302.464463,18530.740029,22348.468150,20536.102179,15101.367063,33649.091133
2,20190301,강북구,309786.315837,9880.497638,5431.196442,7283.578792,9711.192017,10428.617500,9253.645213,11174.490771,...,10341.266958,8887.088354,10767.592033,10659.554071,13129.848975,12195.954863,13948.280575,12592.444421,10638.855375,30880.407996
3,20190301,강서구,555044.440125,21625.743542,9822.468275,12777.132942,15549.024937,21076.736538,22791.684242,26741.089392,...,23469.645408,23905.708221,25440.519171,21428.805467,20888.450108,18942.343763,22770.555250,20690.581062,15652.085742,38324.215004
4,20190301,관악구,509195.308146,12886.056087,6216.729250,10620.215513,22873.001808,36187.228604,27722.778454,22727.666408,...,30955.631613,22186.326571,19370.199787,15729.165508,16818.280333,15561.019837,18278.822604,16596.909963,13704.277483,33160.681883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920,20200831,용산구,282195.691312,7350.836633,3671.808296,5855.027862,8756.274625,11578.237596,12454.761871,15797.500929,...,13215.164192,13371.096483,14573.192642,11732.790233,12154.864700,9585.061400,10160.850412,8913.941400,6390.993550,17570.400875
9921,20200831,은평구,449585.908367,17757.550475,9773.480533,12657.566225,12208.493163,14056.517746,13618.036858,17116.421133,...,16764.031708,16366.941704,19785.116333,17867.123458,19848.485508,17454.331296,19940.218592,17315.623275,13021.462154,36978.869100
9922,20200831,종로구,272909.327746,5103.800896,3087.196362,5747.752154,9377.880946,11337.200433,10870.575971,13471.582858,...,12033.150017,11039.420808,11481.391279,10623.464658,11945.549829,10161.563458,10751.662933,8801.290167,6247.314654,16933.695796
9923,20200831,중구,267535.523779,4219.071792,1738.493367,2951.266488,6801.071229,11150.250613,13155.216217,16970.461437,...,14031.565421,13534.497746,14060.572250,11216.633625,12057.996933,9630.482379,10099.491438,8165.474567,5575.735629,12577.569808


In [ ]:
sh_all.to_csv('생활인구전처리완.csv',encoding='cp949')